2023-10-18 Due to the fact that I am prototyping a lot of code and changing approaches (multiindex column default to long defaul primarily), literate progrmaming style is failing, re. version one of this notebook is somewhat unsalvagable - a testament to why rushing always fails in the end. At some point I'll have to go through and fix it. in the meantime, copy code over and push ON.

In [ ]:
# set up environment

%reload_ext autoreload
%autoreload 2

from dtwalign import dtw
import pandas as pd
from wine_analysis_hplc_uv import definitions
import seaborn as sns
import seaborn.objects as so
from wine_analysis_hplc_uv.notebooks.dtw_methods import DTWNotebookMethods
from wine_analysis_hplc_uv.old_signal_processing.mindex_signal_processing import (
    SignalProcessor,
)
from pybaselines import Baseline
import seaborn as sns
import matplotlib.pyplot as plt
from wine_analysis_hplc_uv.notebooks import eda_by_category_methods

plotter = eda_by_category_methods.Plotting()

scipro = SignalProcessor()

data = pd.read_parquet(definitions.RAW_PARQ_PATH)
data.head()

As we can see, no effect. The smoothing necessary to remove those detected peaks will result in unsatisfactory loss of signal information. Ergo better to use constraints in the peak detection algo. Also, remaining in simple long form with no multiindex massively reduces reshaping overhead and makes UDF functions much simpler to define..

Now lets add kwargs for peak finder.. added.

In [ ]:
# time std and drop vars level
dtwprocess = eda_by_category_methods.DTWProcessing()
sdata = dtwprocess.std_time(data)
sdata.head()

In [ ]:
# smooth
from scipy import signal

smdata = dtwprocess.smooth(sdata, "samplecode", "signal", "smooth_signal")
smdata.head()

In [ ]:
# add bline

bcdata = dtwprocess.blinecorr(
    smdata, "samplecode", "smooth_signal", bcorr_label="bcorr"
)

bcdata.head()

In [ ]:
# plot baseline correction

(
    bcdata.melt(
        id_vars=["samplecode", "wine", "mins"], var_name="siglabel", value_name="sig"
    ).pipe(
        lambda df: so.Plot(data=df, x="mins", y="sig", color="siglabel")
        .facet(col="samplecode", wrap=2)
        .add(mark=so.Line())
    )
)

In [ ]:
# detect peaks prior to alignment

peaks = eda_by_category_methods.Peaks()

pddata = peaks.peakdetect(
    bcdata, grouper="samplecode", signal_label="bcorr", col_label="unaligned_peaks"
)


pddata.head()

In [ ]:
# plot to display peaks
(
    pddata.melt(
        id_vars=["samplecode", "wine", "mins"], var_name="siglabel", value_name="sig"
    )
    .loc[lambda df: (df.mins < 5) & (df.mins > 3)]
    .pivot_table(index=["samplecode", "wine", "mins"], columns="siglabel", values="sig")
    .reset_index()
    .pipe(
        lambda df: so.Plot(data=df, x="mins", color="samplecode")
        .add(so.Line(), y="bcorr")
        .add(so.Dot(), y="unaligned_peaks")
    )
)

For testing purposes, we're only interested in the top 20 peaks

In [ ]:
# get top 20 peaks of the unaligned set

tpdata = peaks.top_peaks(
    pddata, "samplecode", "unaligned_peaks", 20, "top_20_unaligned"
)
tpdata

In [ ]:
# plot the peaks on top of the curves

peak_y = "top_20_unaligned"
display(
    tpdata.loc[lambda df: df.mins < 21].pipe(
        lambda df: (
            so.Plot(df, x="mins", color="samplecode")
            .add(so.Line(), y="bcorr")
            .add(so.Dot(), y=peak_y)
            .plot()
        )
    )
)

# peak table
()

In [ ]:
peaks.peak_table(tpdata, ["samplecode", "wine"], "top_20_unaligned", "peak_n", "mins")

Now align, get the top 10 peaks after alignment and compare them.

Notes: Use transform with a custom function if you are expecting a result with the same shape as the input i.e. a column of the dataframe. Apply rarely gets this right.

In [ ]:
# ref = ApplyDTW().find_ref(tdata, 'samplecode','bcorr')

dtwalign = eda_by_category_methods.ApplyDTW()

# aligned data
adata = dtwalign.align(
    tpdata, primary_key="samplecode", siglabel="bcorr", aligned_label="aligned"
)
adata.head()

In [ ]:
# plot aligned dataset

(
    adata.loc[lambda df: df.mins < 20].pipe(
        lambda df: so.Plot(data=df, x="mins", color="samplecode")
        .layout(size=(5, 3))
        .add(so.Line(), y="aligned")
    )
)

In [ ]:
# plot aligned against unaligned set
(
    adata.loc[lambda df: df.mins < 20]
    .melt(
        id_vars=["mins", "samplecode", "wine"],
        var_name="siglabel",
        value_vars=["bcorr", "aligned"],
        value_name="sig_val",
    )
    .pipe(
        lambda df: so.Plot(data=df, x="mins", color="siglabel")
        .layout(size=(15, 10))
        .add(so.Line(), y="sig_val")
        .facet(col="samplecode", wrap=2)
    )
)

In [ ]:
# detect peaks in aligned set
# aligned peaks data
apddata = peaks.peakdetect(
    adata, "samplecode", signal_label="aligned", col_label="peaks_aligned"
)

apddata.head()

In [ ]:
# add column of top 20 peaks in aligned peak set
# top aligned peaks data
tappdata = apddata.assign(
    aligned_top_20=lambda df: df.groupby("samplecode", group_keys=False)[
        "peaks_aligned"
    ].nlargest(20)
)
tappdata.info()

In [ ]:
# construct and display peak table

aligned_peak_table = tappdata.pipe(
    peaks.peak_table,
    grouper=["samplecode", "wine"],
    peaks_label="aligned_top_20",
    peak_num_label="peak_n",
    peak_idx_label="mins",
)
aligned_peak_table

In [ ]:
# plot overlay signals and top 20 peaks

(
    tappdata.loc[lambda df: df.mins < 20].pipe(
        lambda df: so.Plot(df, x="mins", color="samplecode")
        .add(so.Line(), y="aligned")
        .add(so.Dot(), y="aligned_top_20")
        .facet(col="samplecode", wrap=2)
        .layout(size=(15, 10))
    )
)

In [ ]:
# plot overlay aligned set, unaligned peaks, aligned peaks as dots

plot_peak_data = tappdata.melt(
    id_vars=["mins", "samplecode", "wine"],
    value_vars=["aligned_top_20", "top_20_unaligned"],
    var_name="peak_label",
    value_name="sig",
).dropna()

(
    tappdata.loc[lambda df: df.mins < 20].pipe(
        lambda df: so.Plot(df, x="mins")
        .add(so.Line(), y="aligned", color="samplecode")
        .add(
            so.Dot(),
            data=plot_peak_data.loc[lambda df: df.mins < 20],
            alpha="peak_label",
            color="samplecode",
            x="mins",
            y="sig",
        )
        # .facet(col='samplecode',wrap=2)
        .layout(size=(15, 10))
    )
)

As we can see in the above graphic, The majority of the peaks have lost amplitude, and torbreck-struie has straight up lost a peak at 18 mins. We need a better DTW algorithm.

Need to develop a more reliable alignment method. But we also should look at whether torbreck-struie itself is the issue. First what happens if we exclude it during warping?

Continue this in another book. export the processed data to parquet


In [ ]:
tappdata.loc[:, ["mins", "samplecode", "wine", "signal", "bcorr"]].to_parquet(
    "/Users/jonathan/mres_thesis/wine_analysis_hplc_uv/src/wine_analysis_hplc_uv/notebooks/processed_data.parquet"
)